In [2]:
from pymongo import MongoClient

import time

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client['astro_db']
astro_nodes = db['astro_nodes']
astro_data = db['astro_data']

In [4]:
treeID = 'Dave Large CP Test'

## Get all leaf nodes in a portfolio with include and exclude filtering

In [5]:
# %%timeit

start_time = time.perf_counter()
# include_filter = ['Approved']
# include_filter = ['ProjectStatus:Approved','ProjectStatus:Planning']
# include_filter = ['Portfolio:Insecticides']
# exclude_filter =['Portfolio:Insecticides','Portfolio:Fungicides']
include_filter = []
exclude_filter = []

query = {
        'treeID': treeID,
        'children':[]
}


# Dynamically build the tags filter
tags_filter = {}

if include_filter:  # Add $in only if include_filter is not empty
    # tags_filter["$in"] = [f'ProjectStatus:{entry}' for entry in include_filter]
    tags_filter['$in'] = include_filter

if exclude_filter:  # Add $nin only if exclude_filter is not empty
    tags_filter["$nin"] = exclude_filter

# Only add "tags" filter to query if it has conditions
if tags_filter:
    query["tags"] = tags_filter    
            

# print(query)
projection = {'_id':1, 'data':1, 'name':1, 'path':1, 'children':1, 'isDataValid':1, 'attribute':1}

leaves = list(astro_nodes.find(query,projection))

end_time = time.perf_counter()
print(f"Execution time: {(end_time-start_time):.6f} seconds")
print("Leaves loaded:  ", len(leaves))


Execution time: 2.453315 seconds
Leaves loaded:   30394


### Get all the non-children nodes in a portfolio accounting for tree filtering of the leaf nodes

In [6]:
start_time = time.perf_counter()

pipeline = [
    # Step 1: Match documents based on treeID and empty children
    {"$match": query},
    # {"$match": {"treeID": treeID, "children": []}},

    # Step 2: Unwind the "path" array to get individual elements
    {"$unwind": "$path"},

    # Step 3: Collect unique path elements into a set
    {"$group": {"_id": None, "uniquePaths": {"$addToSet": "$path"}}},

    # Step 4: Lookup all documents where "path" contains any of these uniquePaths
    {
        "$lookup": {
            "from": "astro_nodes",  # The same collection (self-join)
            "localField": "uniquePaths",
            "foreignField": "_id",
            "as": "matching_docs"
        }
    },

    # Step 5: Unwind the "matching_docs" array to get each document separately
    {"$unwind": "$matching_docs"},

    # Step 6: Project only the required fields
    {
        "$project": {
            "_id": "$matching_docs._id",
            "name": "$matching_docs.name",
            "children": "$matching_docs.children",
            "parent": "$matching_docs.parent",
            "data":"$matching_docs.data",
            'isDataValid':'$matching_docs.isDataValid',
            "attribute":"$matching_docs.attribute"
        }
    }
]

# Execute the aggregation
subNodeResults = list(astro_nodes.aggregate(pipeline))

end_time = time.perf_counter()
print(f"Execution time: {(end_time-start_time):.6f} seconds")
print("Nodes loaded:  ", len(subNodeResults))

Execution time: 0.774651 seconds
Nodes loaded:   2913


### Example loading leaves using tree filtering

In [7]:
# %%timeit

start_time = time.perf_counter()
# include_filter = ['Approved']
include_filter = ['ProjectStatus:Approved','ProjectStatus:Planning']
# include_filter = ['Portfolio:Insecticides']
# exclude_filter =['Portfolio:Insecticides','Portfolio:Fungicides']
# include_filter = []
exclude_filter = []

query = {
        'treeID': treeID,
        'children':[]
}


# Dynamically build the tags filter
tags_filter = {}

if include_filter:  # Add $in only if include_filter is not empty
    # tags_filter["$in"] = [f'ProjectStatus:{entry}' for entry in include_filter]
    tags_filter['$in'] = include_filter

if exclude_filter:  # Add $nin only if exclude_filter is not empty
    tags_filter["$nin"] = exclude_filter

# Only add "tags" filter to query if it has conditions
if tags_filter:
    query["tags"] = tags_filter    
            

# print(query)
projection = {'_id':1, 'data':1, 'name':1, 'path':1, 'children':1, 'isDataValid':1, 'attribute':1}

leaves = list(astro_nodes.find(query,projection))

end_time = time.perf_counter()
print(f"Execution time: {(end_time-start_time):.6f} seconds")
print("Leaves loaded:  ", len(leaves))


Execution time: 1.407508 seconds
Leaves loaded:   14984


In [ ]:
### Use same query for loading leaves (with tree filtering)

In [ ]:
start_time = time.perf_counter()

pipeline = [
    # Step 1: Match documents based on treeID and empty children
    {"$match": query},
    # {"$match": {"treeID": treeID, "children": []}},

    # Step 2: Unwind the "path" array to get individual elements
    {"$unwind": "$path"},

    # Step 3: Collect unique path elements into a set
    {"$group": {"_id": None, "uniquePaths": {"$addToSet": "$path"}}},

    # Step 4: Lookup all documents where "path" contains any of these uniquePaths
    {
        "$lookup": {
            "from": "astro_nodes",  # The same collection (self-join)
            "localField": "uniquePaths",
            "foreignField": "_id",
            "as": "matching_docs"
        }
    },

    # Step 5: Unwind the "matching_docs" array to get each document separately
    {"$unwind": "$matching_docs"},

    # Step 6: Project only the required fields
    {
        "$project": {
            "_id": "$matching_docs._id",
            "name": "$matching_docs.name",
            "children": "$matching_docs.children",
            "parent": "$matching_docs.parent",
            "data":"$matching_docs.data",
            'isDataValid':'$matching_docs.isDataValid',
            "attribute":"$matching_docs.attribute"
        }
    }
]

# Execute the aggregation
subNodeResults = list(astro_nodes.aggregate(pipeline))

end_time = time.perf_counter()
print(f"Execution time: {(end_time-start_time):.6f} seconds")
print("Nodes loaded:  ", len(subNodeResults))